# CIFAR-10

La première étape intermédiaire de notre projet est d'utiliser les algorithmes ci-dessous sur le célébre dataset CIFAR-10.

Les algorithmes à étudier :

**Modèles précédents**
- Modèle Linéaire
- Perceptron Multicouches

**Nouveaux modèles**
- Conv Net(s)
- ResNets / HighwayNets - RNN(s)

Pour chacun des algorithmes cités, il faut :
1. L'influence de tous les hyperparamètres des modèles
    - Structure
    - Fonctions d'activations
    - etc.
2. Les paramètres des algorithmes d'apprentissages
    - Learning Rate
    - Momentum
    - etc.

----

In [113]:
import os
import numpy as np
from numpy.random import seed
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Conv2D, BatchNormalization, Input, Average, MaxPool2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.activations import relu, softmax, tanh
from tensorflow.keras.initializers import he_normal, glorot_uniform
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.random import set_seed

In [49]:
print("Version de TensorFlow :", tf.__version__)
print("Nom du GPU :", tf.test.gpu_device_name())

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(False)

Version de TensorFlow : 2.6.0
Nom du GPU : /device:GPU:0


2021-12-19 15:11:19.819920: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-19 15:11:19.820281: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Importation du dataset

In [50]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [51]:
NUM_CLASSES = 10
IMG_SIZE = x_train[0].shape

In [52]:
x_train = x_train.astype('float32') / 256
x_test = x_test.astype('float32') / 256

y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

In [53]:
LOG_DIR = os.path.join("logs")

## Fixer les seeds

In [54]:
set_seed(42) # TensorFlow
seed(42) # NumPy

# Modèle linéaire

In [10]:
EPOCHS = 30
SHUFFLE = True

In [41]:
class Perceptron(Model):
    def __init__(self, activation_function, kernel_initializer, l2_val=0, l1_val=0):
        super(Perceptron, self).__init__()
        self._flatten_layer = Flatten()
        if l1_val > 0 and l2_val > 0:
            self._dense_layer = Dense(IMG_SIZE[0] * IMG_SIZE[1] * IMG_SIZE[2],
                                      activation=activation_function,
                                      kernel_initializer=kernel_initializer,
                                      kernel_regularizer=l1_l2(l1_val, l2_val),
                                      bias_regularizer=l1_l2(l1_val, l2_val)
                                      )
        elif l1_val > 0:
            self._dense_layer = Dense(IMG_SIZE[0] * IMG_SIZE[1] * IMG_SIZE[2],
                                      activation=activation_function,
                                      kernel_initializer=kernel_initializer,
                                      kernel_regularizer=l1(l1_val),
                                      bias_regularizer=l1(l1_val)
                                      )
        elif l2_val > 0:
            self._dense_layer = Dense(IMG_SIZE[0] * IMG_SIZE[1] * IMG_SIZE[2],
                                      activation=activation_function,
                                      kernel_initializer=kernel_initializer,
                                      kernel_regularizer=l2(l2_val),
                                      bias_regularizer=l2(l2_val)
                                      )
        else:
            self._dense_layer = Dense(IMG_SIZE[0] * IMG_SIZE[1] * IMG_SIZE[2],
                                      activation=activation_function,
                                      kernel_initializer=kernel_initializer
                                      )
        self._output_layer = Dense(NUM_CLASSES, activation=softmax)
                
    
    def call(self, inputs):
        x = self._flatten_layer(inputs)
        x = self._dense_layer(x)
        return self._output_layer(x)

In [47]:
EPOCHS = 500
BATCH_SIZE = 512

learning_rate = 0.01
momentum = 0.8
l2_val = 0.01
l1_val = 0.00
using_l1 = l1_val > 0
using_l2 = l2_val > 0
ki = glorot_uniform
fa = tanh
version = '_pft'

perceptron = Perceptron(tanh, glorot_uniform, l1_val=l1_val, l2_val=l2_val)
perceptron.compile(loss=categorical_crossentropy,
                   optimizer=SGD(learning_rate=learning_rate,
                                 momentum=momentum),
                   metrics=categorical_accuracy)

log_name = os.path.join(LOG_DIR,
                            "linear",
                            f"linear_model_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rate}_mo_{momentum}_{('l1_'+str(l1_val)+'_l2_'+str(l2_val) if using_l1 and using_l2 else ('l1_'+str(l1_val) if using_l1 else ('l2_'+str(l2_val) if using_l2 else '')))}_ki_{ki.__name__}_af_{fa.__name__}{version}")

print(log_name)
perceptron.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test, y_test), shuffle=True, callbacks=[TensorBoard(log_name, histogram_freq=1)])

logs/linear/linear_model_ep_500_bs_512_opt_SGD_lr_0.01_mo_0.8_l2_0.01_ki_GlorotUniform_af_tanh_pft


2021-12-18 13:04:24.330486: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-18 13:04:24.330514: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-18 13:04:24.330865: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/500
 3/98 [..............................] - ETA: 3s - loss: 33.1926 - categorical_accuracy: 0.1081 

2021-12-18 13:04:25.003924: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-18 13:04:25.090938: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-18 13:04:25.090947: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-18 13:04:25.135918: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-18 13:04:25.136214: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-18 13:04:25.137301: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/linear/linear_model_ep_500_bs_512_opt_SGD_lr_0.01_mo_0.8_l2_0.01_ki_GlorotUniform_af_tanh_pft/train/plugins/profile/2021_12_18_13_04_25

2021-12-18 13:04:25.137752: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/line

97/98 [============================>.] - ETA: 0s - loss: 30.0960 - categorical_accuracy: 0.3081

...

98/98 [==============================] - 3s 30ms/step - loss: 1.6920 - categorical_accuracy: 0.4407 - val_loss: 1.6870 - val_categorical_accuracy: 0.4373
Epoch 51/500
98/98 [==============================] - 3s 31ms/step - loss: 1.6740 - categorical_accuracy: 0.4464 - val_loss: 1.6730 - val_categorical_accuracy: 0.4496
Epoch 52/500
98/98 [==============================] - 3s 30ms/step - loss: 1.6705 - categorical_accuracy: 0.4473 - val_loss: 1.7652 - val_categorical_accuracy: 0.4083
Epoch 53/500
98/98 [==============================] - 3s 30ms/step - loss: 1.6693 - categorical_accuracy: 0.4496 - val_loss: 1.7307 - val_categorical_accuracy: 0.4177
Epoch 54/500
98/98 [==============================] - 3s 30ms/step - loss: 1.6694 - categorical_accuracy: 0.4509 - val_loss: 1.7008 - val_categorical_accuracy: 0.4389
Epoch 55/500
98/98 [==============================] - 3s 30ms/step - loss: 1.6732 - categorical_accuracy: 0.4461 - val_loss: 1.7417 - val_categorical_accuracy: 0.4179
Epoch 56/50

98/98 [==============================] - 3s 31ms/step - loss: 1.6295 - categorical_accuracy: 0.4656 - val_loss: 1.7321 - val_categorical_accuracy: 0.4358
Epoch 100/500
98/98 [==============================] - 3s 32ms/step - loss: 1.6191 - categorical_accuracy: 0.4698 - val_loss: 1.6506 - val_categorical_accuracy: 0.4472
Epoch 101/500
98/98 [==============================] - 3s 30ms/step - loss: 1.6322 - categorical_accuracy: 0.4627 - val_loss: 1.6551 - val_categorical_accuracy: 0.4572
Epoch 102/500
98/98 [==============================] - 3s 30ms/step - loss: 1.6181 - categorical_accuracy: 0.4706 - val_loss: 1.6703 - val_categorical_accuracy: 0.4490
Epoch 103/500
98/98 [==============================] - 3s 30ms/step - loss: 1.6150 - categorical_accuracy: 0.4712 - val_loss: 1.6304 - val_categorical_accuracy: 0.4645
Epoch 104/500
98/98 [==============================] - 3s 30ms/step - loss: 1.6052 - categorical_accuracy: 0.4773 - val_loss: 1.6294 - val_categorical_accuracy: 0.4621
Epoch 

Epoch 148/500
98/98 [==============================] - 3s 31ms/step - loss: 1.5999 - categorical_accuracy: 0.4810 - val_loss: 1.6292 - val_categorical_accuracy: 0.4603
Epoch 149/500
98/98 [==============================] - 3s 33ms/step - loss: 1.5999 - categorical_accuracy: 0.4807 - val_loss: 1.7108 - val_categorical_accuracy: 0.4311
Epoch 150/500
98/98 [==============================] - 3s 32ms/step - loss: 1.5851 - categorical_accuracy: 0.4853 - val_loss: 1.6043 - val_categorical_accuracy: 0.4758
Epoch 151/500
98/98 [==============================] - 3s 32ms/step - loss: 1.5811 - categorical_accuracy: 0.4873 - val_loss: 1.6212 - val_categorical_accuracy: 0.4695
Epoch 152/500
98/98 [==============================] - 3s 30ms/step - loss: 1.5928 - categorical_accuracy: 0.4797 - val_loss: 1.6386 - val_categorical_accuracy: 0.4498
Epoch 153/500
98/98 [==============================] - 3s 32ms/step - loss: 1.5972 - categorical_accuracy: 0.4785 - val_loss: 1.6247 - val_categorical_accuracy:

Epoch 197/500
98/98 [==============================] - 5s 47ms/step - loss: 1.5710 - categorical_accuracy: 0.4895 - val_loss: 1.6382 - val_categorical_accuracy: 0.4679
Epoch 198/500
98/98 [==============================] - 4s 46ms/step - loss: 1.5737 - categorical_accuracy: 0.4926 - val_loss: 1.6448 - val_categorical_accuracy: 0.4669
Epoch 199/500
98/98 [==============================] - 4s 45ms/step - loss: 1.5622 - categorical_accuracy: 0.4935 - val_loss: 1.6021 - val_categorical_accuracy: 0.4750
Epoch 200/500
98/98 [==============================] - 4s 43ms/step - loss: 1.5935 - categorical_accuracy: 0.4839 - val_loss: 1.6212 - val_categorical_accuracy: 0.4765
Epoch 201/500
98/98 [==============================] - 4s 40ms/step - loss: 1.5755 - categorical_accuracy: 0.4926 - val_loss: 1.6308 - val_categorical_accuracy: 0.4730
Epoch 202/500
98/98 [==============================] - 4s 39ms/step - loss: 1.5574 - categorical_accuracy: 0.4986 - val_loss: 1.5811 - val_categorical_accuracy:

KeyboardInterrupt: 

## Trouver le meilleur modèle

In [9]:
def linear_model(activation_function, kernel_initializer, learning_rate, momentum, batch_size, version=''):
    input_layer = Input(shape=(32, 32, 3))

    hidden_layers = Flatten()(input_layer)
    hidden_layers = Dense(IMG_SIZE[0] * IMG_SIZE[1] * IMG_SIZE[2], activation=activation_function, kernel_initializer=kernel_initializer)(hidden_layers)

    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    linear_model = Model(input_layer, output_layer)
    
    linear_model.compile(loss=categorical_crossentropy,
                         optimizer=SGD(learning_rate=learning_rate,
                                       momentum=momentum),
                         metrics=categorical_accuracy)
    log_name = os.path.join(LOG_DIR,
                            "linear",
                            f"linear_model_ep_{EPOCHS}_bs_{batch_size}_opt_SGD_lr_{learning_rate}_mo_{momentum}_ki_{kernel_initializer.__name__}_af_{activation_function.__name__}{version}")
    history = linear_model.fit(x_train,
                               y_train,
                               batch_size=batch_size,
                               epochs=EPOCHS,
                               validation_data=(x_test, y_test),
                               shuffle=SHUFFLE,
                               callbacks=[TensorBoard(log_name, histogram_freq=1)])
    return linear_model, history

Les hyperparamètres que nous souhaitons ajuster :

In [10]:
learning_rates = [0.01, 0.05, 0.1, 0.2]
kernel_initializers = [glorot_uniform, he_normal]
activation_functions = [tanh, relu]
batch_sizes = [64, 128, 256]
momentums = [0, 0.5, 0.9]

In [ ]:
# Grid Search V1

best_model_accuracy = 0
counter = 0
linear_models = {}

for mo in momentums:
    for bs in batch_sizes:
        for af in activation_functions:
            for ki in kernel_initializers:
                for lr in learning_rates:
                    print(f"Iteration n°{counter} | af: {af.__name__} - ki: {ki.__name__} - SGD lr: {lr} / mo: {mo} - bs: {bs}")
                    model_name = f"af_{af.__name__}_ki_{ki.__name__}_sgd_lr_{lr}_mo_{mo}_bs_{bs}"
                    model, history = linear_model(af, ki, lr, mo, bs)
                    accuracy = round(history.history["categorical_accuracy"][-1], 3)
                    print(f"Accuracy : {accuracy}")
                    linear_models[model_name] = accuracy
                    if accuracy > best_model_accuracy:
                        model.save(f"models/linear/{str(accuracy)+'_'+model_name}.keras")
                        best_model_accuracy = accuracy
                    counter += 1

Après une trentaine d'épochs, nous avons remarqué que nous avons les pires résultats lorsque nous utilisons un learning rate égale à 0.1 et 0.2. Du coup, nous allons les supprimer lors de la prochaine version. Nous allons aussi en profiter pour accrocher les fonctions d'activations à l'initialisateur de kernel conseillé durant le cours (tanh -> glorot_uniform et relu -> he_normal)

**Résultat:**

Les meilleurs modèles que nous avons ont en commun un batch size de 64, un learning rate de 0.05, peu de momentum et enfin le combo relu/he_normal. 

In [12]:
#best_models_sorted = sorted(linear_models.items(), key=lambda x: x[1], reverse=True)
#print(best_models_sorted[:5])

In [13]:
best_linear_model = load_model('models/linear/af_relu_ki_HeNormal_sgd_lr_0.05_mo_0_bs_64_0.708.keras')

2021-12-17 10:37:14.580906: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-17 10:37:14.580954: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [14]:
test_loss, test_accuracy = best_linear_model.evaluate(x_test, y_test)
print(f"Test loss : {round(test_loss, 2)}")
print(f"Test accuracy : {round(test_accuracy, 2) * 100}%")

2021-12-17 10:37:17.547261: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-17 10:37:17.550501: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-17 10:37:17.604341: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 2s 4ms/step - loss: 1.5417 - categorical_accuracy: 0.5075
Test loss : 1.54
Test accuracy : 51.0%


Pour l'instant, notre meilleur modèle prédit correctement 51% du temps, ce qui est pas mal sur 10 catégories.
Nous allons donc continuer de chercher le meilleure modèle en restant proche de ces paramètres, nous allons également ajouter à nos hyperparamètres, une forme de régularisation (norme l1, l2).

In [28]:
EPOCHS = 500

In [29]:
learning_rates = [0.04, 0.05, 0.06]
momentums = [0, 0.10, 0.25]
batch_size = 512
kernel_initializer = he_normal
activation_function = relu

In [30]:
# Panda fine-tuning
learning_rate = 0.035
momentum = 0.5
l2_val = 0.01


input_layer = Input(shape=(32, 32, 3))

hidden_layers = Flatten()(input_layer)
hidden_layers = Dense(IMG_SIZE[0] * IMG_SIZE[1] * IMG_SIZE[2], activation=activation_function, kernel_initializer=kernel_initializer)(hidden_layers)

output_layer = Dense(NUM_CLASSES, activation=softmax,
                     kernel_regularizer = l2(l2_val),
                     bias_regularizer = l2(l2_val))(hidden_layers)
linear_model = Model(input_layer, output_layer)

linear_model.compile(loss=categorical_crossentropy,
                     optimizer=SGD(learning_rate=learning_rate,
                                   momentum=momentum),
                     metrics=categorical_accuracy)
log_name = os.path.join(LOG_DIR,
                        "linear",
                        f"linear_model_ep_{EPOCHS}_bs_{batch_size}_opt_SGD_lr_{learning_rate}_mo_{momentum}_ki_{kernel_initializer.__name__}_af_{activation_function.__name__}_l2_{l2_val}_pft")
history = linear_model.fit(x_train,
                           y_train,
                           batch_size=batch_size,
                           epochs=EPOCHS,
                           validation_data=(x_test, y_test),
                           shuffle=SHUFFLE,
                           callbacks=[TensorBoard(log_name, histogram_freq=1)])
accuracy = round(history.history["val_categorical_accuracy"][-1], 3)
print(f"Accuracy : {accuracy}")

2021-12-17 11:08:07.889242: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-17 11:08:07.889262: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-17 11:08:07.889370: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/500
 3/98 [..............................] - ETA: 3s - loss: 3.4384 - categorical_accuracy: 0.1009 

2021-12-17 11:08:08.537147: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-17 11:08:08.612097: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-17 11:08:08.612106: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-17 11:08:08.649351: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-17 11:08:08.649644: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-17 11:08:08.650689: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/linear/linear_model_ep_500_bs_512_opt_SGD_lr_0.035_mo_0.5_ki_HeNormal_af_relu_l2_0.01_pft/train/plugins/profile/2021_12_17_11_08_08

2021-12-17 11:08:08.651127: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/linear/l

97/98 [============================>.] - ETA: 0s - loss: 2.2165 - categorical_accuracy: 0.3292

2021-12-17 11:08:10.966469: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


...

Epoch 295/500
98/98 [==============================] - 2s 24ms/step - loss: 1.0344 - categorical_accuracy: 0.7277 - val_loss: 1.4556 - val_categorical_accuracy: 0.5443
Epoch 296/500
98/98 [==============================] - 2s 23ms/step - loss: 1.0316 - categorical_accuracy: 0.7304 - val_loss: 1.4200 - val_categorical_accuracy: 0.5587
Epoch 297/500
98/98 [==============================] - 2s 25ms/step - loss: 1.0303 - categorical_accuracy: 0.7301 - val_loss: 1.4108 - val_categorical_accuracy: 0.5625
Epoch 298/500
98/98 [==============================] - 2s 25ms/step - loss: 1.0316 - categorical_accuracy: 0.7282 - val_loss: 1.4282 - val_categorical_accuracy: 0.5581
Epoch 299/500
98/98 [==============================] - 2s 24ms/step - loss: 1.0279 - categorical_accuracy: 0.7322 - val_loss: 1.4071 - val_categorical_accuracy: 0.5638
Epoch 300/500
98/98 [==============================] - 2s 24ms/step - loss: 1.0284 - categorical_accuracy: 0.7296 - val_loss: 1.4291 - val_categorical_accuracy:

Epoch 344/500
98/98 [==============================] - 2s 24ms/step - loss: 0.9900 - categorical_accuracy: 0.7489 - val_loss: 1.4146 - val_categorical_accuracy: 0.5645
Epoch 345/500
98/98 [==============================] - 2s 24ms/step - loss: 0.9902 - categorical_accuracy: 0.7477 - val_loss: 1.4234 - val_categorical_accuracy: 0.5610
Epoch 346/500
98/98 [==============================] - 2s 24ms/step - loss: 0.9897 - categorical_accuracy: 0.7487 - val_loss: 1.4173 - val_categorical_accuracy: 0.5637
Epoch 347/500
98/98 [==============================] - 2s 24ms/step - loss: 0.9888 - categorical_accuracy: 0.7484 - val_loss: 1.4234 - val_categorical_accuracy: 0.5595
Epoch 348/500
98/98 [==============================] - 2s 24ms/step - loss: 0.9890 - categorical_accuracy: 0.7479 - val_loss: 1.4259 - val_categorical_accuracy: 0.5612
Epoch 349/500
98/98 [==============================] - 2s 23ms/step - loss: 0.9851 - categorical_accuracy: 0.7519 - val_loss: 1.4447 - val_categorical_accuracy:

Epoch 393/500
98/98 [==============================] - 2s 25ms/step - loss: 0.9522 - categorical_accuracy: 0.7666 - val_loss: 1.4082 - val_categorical_accuracy: 0.5697
Epoch 394/500
98/98 [==============================] - 3s 27ms/step - loss: 0.9521 - categorical_accuracy: 0.7659 - val_loss: 1.4119 - val_categorical_accuracy: 0.5683
Epoch 395/500
98/98 [==============================] - 2s 25ms/step - loss: 0.9532 - categorical_accuracy: 0.7668 - val_loss: 1.4186 - val_categorical_accuracy: 0.5644
Epoch 396/500
98/98 [==============================] - 3s 26ms/step - loss: 0.9500 - categorical_accuracy: 0.7691 - val_loss: 1.4271 - val_categorical_accuracy: 0.5613
Epoch 397/500
98/98 [==============================] - 3s 26ms/step - loss: 0.9480 - categorical_accuracy: 0.7690 - val_loss: 1.4314 - val_categorical_accuracy: 0.5602
Epoch 398/500
98/98 [==============================] - 3s 27ms/step - loss: 0.9508 - categorical_accuracy: 0.7673 - val_loss: 1.4366 - val_categorical_accuracy:

Epoch 442/500
98/98 [==============================] - 2s 24ms/step - loss: 0.9177 - categorical_accuracy: 0.7832 - val_loss: 1.4219 - val_categorical_accuracy: 0.5655
Epoch 443/500
98/98 [==============================] - 2s 24ms/step - loss: 0.9172 - categorical_accuracy: 0.7836 - val_loss: 1.4566 - val_categorical_accuracy: 0.5556
Epoch 444/500
98/98 [==============================] - 3s 26ms/step - loss: 0.9162 - categorical_accuracy: 0.7846 - val_loss: 1.4287 - val_categorical_accuracy: 0.5646
Epoch 445/500
98/98 [==============================] - 2s 25ms/step - loss: 0.9133 - categorical_accuracy: 0.7872 - val_loss: 1.4328 - val_categorical_accuracy: 0.5603
Epoch 446/500
98/98 [==============================] - 3s 26ms/step - loss: 0.9119 - categorical_accuracy: 0.7873 - val_loss: 1.4312 - val_categorical_accuracy: 0.5676
Epoch 447/500
98/98 [==============================] - 3s 27ms/step - loss: 0.9128 - categorical_accuracy: 0.7871 - val_loss: 1.4305 - val_categorical_accuracy:

Epoch 491/500
98/98 [==============================] - 2s 23ms/step - loss: 0.8861 - categorical_accuracy: 0.7983 - val_loss: 1.4441 - val_categorical_accuracy: 0.5626
Epoch 492/500
98/98 [==============================] - 2s 23ms/step - loss: 0.8849 - categorical_accuracy: 0.7986 - val_loss: 1.4338 - val_categorical_accuracy: 0.5683
Epoch 493/500
98/98 [==============================] - 2s 23ms/step - loss: 0.8861 - categorical_accuracy: 0.7990 - val_loss: 1.4424 - val_categorical_accuracy: 0.5655
Epoch 494/500
98/98 [==============================] - 2s 23ms/step - loss: 0.8836 - categorical_accuracy: 0.8001 - val_loss: 1.4283 - val_categorical_accuracy: 0.5710
Epoch 495/500
98/98 [==============================] - 2s 25ms/step - loss: 0.8832 - categorical_accuracy: 0.7987 - val_loss: 1.4280 - val_categorical_accuracy: 0.5729
Epoch 496/500
98/98 [==============================] - 2s 24ms/step - loss: 0.8851 - categorical_accuracy: 0.7985 - val_loss: 1.4426 - val_categorical_accuracy:

In [ ]:
#model_name = f"af_{activation_function.__name__}_ki_{kernel_initializer.__name__}_sgd_lr_{0.05}_mo_{0.10}_bs_{batch_size}"
#linear_model.save(f"models/linear/{str(accuracy)+'_'+model_name}.keras")

In [163]:
# Grid Search V2

best_model_accuracy = 0
counter = 0
linear_models = {}

for lr in learning_rates:
    for mo in momentums:
        counter += 1
        print(f"Iteration n°{counter} | af: {activation_function.__name__} - ki: {kernel_initializer.__name__} - SGD lr: {lr} / mo: {mo} - bs: {batch_size}")
        model_name = f"af_{activation_function.__name__}_ki_{kernel_initializer.__name__}_sgd_lr_{lr}_mo_{mo}_bs_{batch_size}"
        model, history = linear_model(activation_function, kernel_initializer, lr, mo, batch_size, version='_v2')
        accuracy = round(history.history["val_categorical_accuracy"][-1], 3)
        print(f"Accuracy : {accuracy}")
        linear_models[model_name] = accuracy
        if accuracy > best_model_accuracy:
            model.save(f"models/linear/{str(accuracy)+'_'+model_name}.keras")
            best_model_accuracy = accuracy

Iteration n°1 | af: relu - ki: HeNormal - SGD lr: 0.04 / mo: 0 - bs: 64


2021-12-17 00:17:58.827727: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-17 00:17:58.827742: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-17 00:17:58.827824: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/50
  7/782 [..............................] - ETA: 14s - loss: 5.4403 - categorical_accuracy: 0.0960

2021-12-17 00:17:59.926755: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-17 00:18:00.002039: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-17 00:18:00.002052: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-17 00:18:00.029046: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-17 00:18:00.029373: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-17 00:18:00.030466: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/linear/linear_model_ep_50_bs_64_opt_SGD_lr_0.04_mo_0_ki_HeNormal_af_relu_v2/train/plugins/profile/2021_12_17_00_18_00

2021-12-17 00:18:00.030903: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/linear/linear_model_ep

779/782 [============================>.] - ETA: 0s - loss: 1.8277 - categorical_accuracy: 0.3605

...

2021-12-17 00:26:33.724586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-17 00:26:33.810521: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-17 00:26:33.810529: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-17 00:26:33.836342: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-17 00:26:33.836642: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-17 00:26:33.837645: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/linear/linear_model_ep_50_bs_64_opt_SGD_lr_0.04_mo_0.1_ki_HeNormal_af_relu_v2/train/plugins/profile/2021_12_17_00_26_33

2021-12-17 00:26:33.838088: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/linear/linear_model_

780/782 [============================>.] - ETA: 0s - loss: 1.8155 - categorical_accuracy: 0.3623

2021-12-17 00:26:45.032421: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 12s 15ms/step - loss: 1.8155 - categorical_accuracy: 0.3623 - val_loss: 1.7611 - val_categorical_accuracy: 0.3611
Epoch 2/50
782/782 [==============================] - 12s 15ms/step - loss: 1.6165 - categorical_accuracy: 0.4278 - val_loss: 1.7516 - val_categorical_accuracy: 0.3535
Epoch 3/50
782/782 [==============================] - 12s 15ms/step - loss: 1.5368 - categorical_accuracy: 0.4601 - val_loss: 1.5994 - val_categorical_accuracy: 0.4249
Epoch 4/50
782/782 [==============================] - 11s 15ms/step - loss: 1.4816 - categorical_accuracy: 0.4804 - val_loss: 1.5883 - val_categorical_accuracy: 0.4224
Epoch 5/50
782/782 [==============================] - 11s 14ms/step - loss: 1.4353 - categorical_accuracy: 0.4980 - val_loss: 1.6067 - val_categorical_accuracy: 0.4298
Epoch 6/50
782/782 [==============================] - 11s 14ms/step - loss: 1.3964 - categorical_accuracy: 0.5096 - val_loss: 1.7055 - val_categorical_accuracy: 0.4037
Epo

KeyboardInterrupt: 

## MLP

In [64]:
NUM_LAYERS = list(range(2, 6))
NUM_UNITS = [8, 16, 32, 64, 128]
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True

In [94]:
learning_rates = [0.02, 0.07, 0.1]
momentums = [0.25, 0.5, 0.9]

def MLP(num_layer, nodes_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = Flatten()(input_layer)
    for n, _ in enumerate(range(num_layer)):
        hidden_layers = Dense(nodes_by_layers[n], activation=relu, kernel_initializer=he_normal)(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)

for lr in learning_rates:
    for mom in momentums:
        num_layer = int(np.random.choice(NUM_LAYERS, 1))
        nodes_by_layers = np.random.choice(NUM_UNITS, num_layer)

        mlp = MLP(num_layer, nodes_by_layers)
        mlp.compile(loss=categorical_crossentropy,
                    optimizer=SGD(lr, momentum=mom),
                    metrics=categorical_accuracy)
        MLP_LOG = os.path.join(LOG_DIR, "mlp",
                        f"mlp_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in nodes_by_layers)}")
        
        mlp.fit(x_train,
                        y_train,
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=(x_test, y_test),
                        shuffle=SHUFFLE,
                        callbacks=[tf.keras.callbacks.TensorBoard(MLP_LOG, histogram_freq=1)]
                       )

2021-12-19 15:48:25.707201: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-19 15:48:25.707232: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-19 15:48:25.707293: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/200


2021-12-19 15:48:26.689126: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


17/98 [====>.........................] - ETA: 0s - loss: 2.3182 - categorical_accuracy: 0.1182

2021-12-19 15:48:27.031010: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-19 15:48:27.031019: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-19 15:48:27.052027: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-19 15:48:27.052389: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-19 15:48:27.053683: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/mlp/mlp_ep_200_bs_512_opt_SGD_lr_0.02_mom_0.25_layers_128_8_128/train/plugins/profile/2021_12_19_15_48_27

2021-12-19 15:48:27.054166: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/mlp/mlp_ep_200_bs_512_opt_SGD_lr_0.02_mom_0.25_layers_128_8_128/train/plugins/profile/2021_12_19_15_48_27/MacBook-Pro-de-Reda.local.trace.json.gz
2021-12-19 15:48:27.054624: I tensorflow/co

...

98/98 [==============================] - 1s 12ms/step - loss: 2.0708 - categorical_accuracy: 0.2461 - val_loss: 1.9395 - val_categorical_accuracy: 0.3059


2021-12-19 15:51:36.737703: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 2/200
98/98 [==============================] - 1s 10ms/step - loss: 1.8873 - categorical_accuracy: 0.3245 - val_loss: 1.8401 - val_categorical_accuracy: 0.3364
Epoch 3/200
98/98 [==============================] - 1s 8ms/step - loss: 1.8087 - categorical_accuracy: 0.3531 - val_loss: 1.7663 - val_categorical_accuracy: 0.3740
Epoch 4/200
98/98 [==============================] - 1s 9ms/step - loss: 1.7658 - categorical_accuracy: 0.3687 - val_loss: 1.7448 - val_categorical_accuracy: 0.3770
Epoch 5/200
98/98 [==============================] - 1s 10ms/step - loss: 1.7199 - categorical_accuracy: 0.3819 - val_loss: 1.6862 - val_categorical_accuracy: 0.3993
Epoch 6/200
98/98 [==============================] - 1s 9ms/step - loss: 1.6987 - categorical_accuracy: 0.3940 - val_loss: 1.6944 - val_categorical_accuracy: 0.3985
Epoch 7/200
98/98 [==============================] - 1s 9ms/step - loss: 1.6741 - categorical_accuracy: 0.4020 - val_loss: 1.6798 - val_categorical_accuracy: 0.4110
Epoch 8/

98/98 [==============================] - 1s 9ms/step - loss: 1.3023 - categorical_accuracy: 0.5353 - val_loss: 1.4585 - val_categorical_accuracy: 0.4763
Epoch 52/200
98/98 [==============================] - 1s 9ms/step - loss: 1.3170 - categorical_accuracy: 0.5307 - val_loss: 1.4269 - val_categorical_accuracy: 0.4983
Epoch 53/200
98/98 [==============================] - 1s 9ms/step - loss: 1.3088 - categorical_accuracy: 0.5329 - val_loss: 1.4273 - val_categorical_accuracy: 0.4925
Epoch 54/200
98/98 [==============================] - 1s 9ms/step - loss: 1.2930 - categorical_accuracy: 0.5401 - val_loss: 1.4246 - val_categorical_accuracy: 0.4949
Epoch 55/200
98/98 [==============================] - 1s 9ms/step - loss: 1.2985 - categorical_accuracy: 0.5380 - val_loss: 1.4377 - val_categorical_accuracy: 0.4875
Epoch 56/200
98/98 [==============================] - 1s 8ms/step - loss: 1.2896 - categorical_accuracy: 0.5422 - val_loss: 1.4454 - val_categorical_accuracy: 0.4863
Epoch 57/200
98/9

KeyboardInterrupt: 

# ConvNet

In [110]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True

In [115]:
learning_rates = [0.02, 0.07, 0.1]
momentums = [0.25, 0.5, 0.9]


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="same", activation=tanh)(hidden_layers)
        hidden_layers = MaxPool2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)

for lr in learning_rates:
    for mom in momentums:
        num_layer = int(np.random.choice(NUM_LAYERS, 1))
        filters_by_layers = [32*x for x in range(1, num_layer+1)]

        convnet = ConvNet(num_layer, filters_by_layers)
        convnet.compile(loss=categorical_crossentropy,
                    optimizer=SGD(lr, momentum=mom),
                    metrics=categorical_accuracy)
        CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
        convnet.fit(x_train,
                        y_train,
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=(x_test, y_test),
                        shuffle=SHUFFLE,
                        callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)]
                       )
                       

2021-12-19 16:51:22.637952: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-19 16:51:22.637966: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-19 16:51:22.638019: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/200


2021-12-19 16:51:23.394278: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


 3/98 [..............................] - ETA: 9s - loss: 2.3047 - categorical_accuracy: 0.0827

2021-12-19 16:51:24.756383: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-19 16:51:24.756394: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-19 16:51:24.856829: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-19 16:51:24.857152: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-19 16:51:24.858286: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/convnet/convnet_ep_200_bs_512_opt_SGD_lr_0.02_mom_0.25_layers_32_64_96_128_160_af_tanh/train/plugins/profile/2021_12_19_16_51_24

2021-12-19 16:51:24.858695: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/convnet/convnet_ep_200_bs_512_opt_SGD_lr_0.02_mom_0.25_layers_32_64_96_128_160_af_tanh/train/plugins/profile/2021_12_19_16_51_24/MacBook-Pro-de-Reda.local.trace.json.

98/98 [==============================] - ETA: 0s - loss: 2.2586 - categorical_accuracy: 0.1735

2021-12-19 16:51:32.729131: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


98/98 [==============================] - 10s 87ms/step - loss: 2.2586 - categorical_accuracy: 0.1735 - val_loss: 2.2043 - val_categorical_accuracy: 0.2423
Epoch 2/200
98/98 [==============================] - 8s 79ms/step - loss: 2.0886 - categorical_accuracy: 0.2888 - val_loss: 1.9503 - val_categorical_accuracy: 0.3301
Epoch 3/200
98/98 [==============================] - 8s 77ms/step - loss: 1.9104 - categorical_accuracy: 0.3224 - val_loss: 1.9221 - val_categorical_accuracy: 0.3029
Epoch 4/200
98/98 [==============================] - 8s 82ms/step - loss: 1.8247 - categorical_accuracy: 0.3426 - val_loss: 1.7549 - val_categorical_accuracy: 0.3705
Epoch 5/200
98/98 [==============================] - 8s 82ms/step - loss: 1.7587 - categorical_accuracy: 0.3645 - val_loss: 1.7019 - val_categorical_accuracy: 0.3845
Epoch 6/200
98/98 [==============================] - 8s 79ms/step - loss: 1.7042 - categorical_accuracy: 0.3838 - val_loss: 1.6686 - val_categorical_accuracy: 0.3935
Epoch 7/200
98/

98/98 [==============================] - 8s 78ms/step - loss: 0.7706 - categorical_accuracy: 0.7328 - val_loss: 0.8906 - val_categorical_accuracy: 0.6880
Epoch 51/200
98/98 [==============================] - 8s 77ms/step - loss: 0.7568 - categorical_accuracy: 0.7397 - val_loss: 0.8933 - val_categorical_accuracy: 0.6897
Epoch 52/200
98/98 [==============================] - 8s 78ms/step - loss: 0.7481 - categorical_accuracy: 0.7428 - val_loss: 0.8842 - val_categorical_accuracy: 0.6852
Epoch 53/200
98/98 [==============================] - 8s 78ms/step - loss: 0.7402 - categorical_accuracy: 0.7441 - val_loss: 0.8886 - val_categorical_accuracy: 0.6867
Epoch 54/200
98/98 [==============================] - 8s 78ms/step - loss: 0.7236 - categorical_accuracy: 0.7498 - val_loss: 0.8804 - val_categorical_accuracy: 0.6912
Epoch 55/200
98/98 [==============================] - 8s 84ms/step - loss: 0.7173 - categorical_accuracy: 0.7541 - val_loss: 0.9224 - val_categorical_accuracy: 0.6753
Epoch 56/20

KeyboardInterrupt: 

# ResNet

In [116]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True

In [121]:
learning_rates = [0.02, 0.07, 0.1]
momentums = [0.25, 0.5, 0.9]


def ResNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        prev_layer = hidden_layers
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding='same', activation=relu,
                                            kernel_initializer=he_normal)(hidden_layers)
        #hidden_layers = Dropout(0.2)(hidden_layers)
        if n > 0:
            hidden_layers = Average()([hidden_layers, prev_layer])
        else:
            hidden_layers = Average()([hidden_layers, Dense(32)(prev_layer)])
    
    hidden_layers = Flatten()(hidden_layers)   
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)

for lr in learning_rates:
    for mom in momentums:
        num_layer = int(np.random.choice(NUM_LAYERS, 1))
        filters_by_layers = [32 for x in range(1, num_layer+1)]

        resnet = ResNet(num_layer, filters_by_layers)
        resnet.compile(loss=categorical_crossentropy,
                    optimizer=SGD(lr, momentum=mom),
                    metrics=categorical_accuracy)
        RESNET_LOG = os.path.join(LOG_DIR, "resnet",
                        f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu")
        resnet.fit(x_train,
                        y_train,
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=(x_test, y_test),
                        shuffle=SHUFFLE,
                        callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)]
                       )
                       

2021-12-19 17:01:57.606703: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-19 17:01:57.606856: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-19 17:01:57.607374: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
/Users/redamaizate/miniforge3/envs/cours_env/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
2021-12-19 17:01:58.789158: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/200
 1/98 [..............................] - ETA: 2:20 - loss: 2.2979 - categorical_accuracy: 0.1289

2021-12-19 17:02:00.092889: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-19 17:02:00.092900: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


 2/98 [..............................] - ETA: 23s - loss: 2.3021 - categorical_accuracy: 0.1279 

2021-12-19 17:02:00.353738: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-19 17:02:00.357145: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-19 17:02:00.361354: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/resnet/resnet_ep_200_bs_512_opt_SGD_lr_0.02_mom_0.25_layers_32_32_32_32_32_af_relu/train/plugins/profile/2021_12_19_17_02_00

2021-12-19 17:02:00.362273: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/resnet/resnet_ep_200_bs_512_opt_SGD_lr_0.02_mom_0.25_layers_32_32_32_32_32_af_relu/train/plugins/profile/2021_12_19_17_02_00/MacBook-Pro-de-Reda.local.trace.json.gz
2021-12-19 17:02:00.366658: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/resnet/resnet_ep_200_bs_512_opt_SGD_lr_0.02_mom_0.25_layers_32_32_32_32_32_af_relu/train/plugins/profile/2021_12_19_17

98/98 [==============================] - ETA: 0s - loss: 2.1314 - categorical_accuracy: 0.2270

2021-12-19 17:02:18.908965: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


98/98 [==============================] - 22s 212ms/step - loss: 2.1314 - categorical_accuracy: 0.2270 - val_loss: 2.0337 - val_categorical_accuracy: 0.2997
Epoch 2/200
98/98 [==============================] - 20s 206ms/step - loss: 1.9130 - categorical_accuracy: 0.3304 - val_loss: 1.8485 - val_categorical_accuracy: 0.3420
Epoch 3/200
98/98 [==============================] - 21s 211ms/step - loss: 1.7972 - categorical_accuracy: 0.3718 - val_loss: 1.9520 - val_categorical_accuracy: 0.3156
Epoch 4/200
98/98 [==============================] - 20s 204ms/step - loss: 1.7322 - categorical_accuracy: 0.3963 - val_loss: 1.6697 - val_categorical_accuracy: 0.4106
Epoch 5/200
98/98 [==============================] - 20s 205ms/step - loss: 1.6770 - categorical_accuracy: 0.4139 - val_loss: 1.6319 - val_categorical_accuracy: 0.4241
Epoch 6/200
98/98 [==============================] - 20s 200ms/step - loss: 1.6437 - categorical_accuracy: 0.4246 - val_loss: 1.6178 - val_categorical_accuracy: 0.4277
Epoc

KeyboardInterrupt: 

# RNN